In [12]:
#!pip install catboost

In [13]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

In [14]:
train = pd.read_csv('./data/ready_train.csv')
test = pd.read_csv('./data/ready_test.csv')
data = pd.read_csv('./data/preprocessed_full.csv')

train_80 = train.sample(frac = 0.8, random_state = 42)
 
# Creating dataframe with 
# rest of the 80% values
train_hold = train.drop(train_80.index)

test_hold = test.drop(train_80.index)


# Straightforward solutions
Let's assume, that the best method for each time series is the method with the highest number of best results.
The accuarcy we get from choosing that method for every time series and the average RMSSE for specified model.

In [15]:
def straightforward(raw_data, best_data, criteria, models):
    baseline = pd.DataFrame(index=criteria,
                            columns=['metric', 'model', 'accuracy', 'lost_rate', 'RMSSE'])
    assert np.array(models).shape == np.array(criteria).shape
    baseline['metric'] = criteria
    baseline['model'] = models

    total = len(best_data)
    for criterion, model in zip(criteria, models):
        # Calculate accuarcy
        tp = best_data[best_data[criterion+'_model']==model].count().iloc[0]
        
        true_results = raw_data[raw_data['model_name']==model]
        n = 0; mae = 0; lost = 0; rmsse = 0
        # For each time series calculate discrepancy between best result and
        # predicted method
        for ts, result in zip(best_data['naming_orig'], best_data[criterion]):
            batch = true_results[true_results['naming_orig']==ts]
            try:
                true_value = batch.iloc[batch[criterion].argmin()][criterion]
                rmsse = rmsse + true_value
                if true_value != result:
                    n = n + 1
                    mae = mae + np.abs(true_value - result)
            except:
                lost = lost + 1
            
        #baseline.loc[criterion, 'MAE'] = mae / n
        baseline.loc[criterion, 'lost_rate'] = lost / total
        baseline.loc[criterion, 'accuracy'] = tp / total
        baseline.loc[criterion, 'RMSSE'] = rmsse / total
        baseline.index = np.arange(len(criteria))
            
    return baseline

## Experiment I
We take the most frequent model from the **train** set and apply it to the **train** holdout as a prediction.

In [16]:
best_train_model = train_80['RMSSE_model'].mode().to_numpy()[0]
train_baseline = straightforward(data[data['split']=='validation'],
                                 train_hold, ['RMSSE'], [best_train_model])
train_baseline

,metric,model,accuracy,lost_rate,RMSSE
0,RMSSE,TFTTuningObjective_gl,0.149425,0.109195,0.839198


## Experiment II
We take the most frequent model from the **train** set and apply it to the **test** holdout as a prediction.

In [17]:
cv_baseline = straightforward(data[data['split']=='test'],
                              test_hold, ['RMSSE'], [best_train_model])
cv_baseline

,metric,model,accuracy,lost_rate,RMSSE
0,RMSSE,TFTTuningObjective_gl,0.132184,0.109195,0.965092


## Experiment III
We take models from the **train** holdout and apply them to the **test** holdout as a prediction

In [18]:
def validation2test(raw_data, val_data, test_data, criteria):
    result = pd.DataFrame(index=criteria,
                          columns=['metric', 'model', 'accuracy', 'lost_rate', 'RMSSE'])
    assert np.array(val_data).shape == np.array(test_data).shape
    result['metric'] = criteria
    result['model'] = 'N/A'
    
    total = len(val_data)
    for criterion in criteria:
        # Calculate accuarcy
        tp = val_data[val_data[criterion+'_model']==test_data[criterion+'_model']].count().iloc[0]
        
        n = 0; mae = 0; lost = 0; rmsse = 0
        
        for ts, metric in zip(test_data['naming_orig'], test_data[criterion]):
            batch = raw_data[raw_data['naming_orig']==ts]
            batch_val = val_data[val_data['naming_orig']==ts]
            value = batch[batch['model_name']==batch_val[criterion+'_model'].values[0]]
            try:
                true_value = value.iloc[value[criterion].argmin()][criterion]
                rmsse = rmsse + true_value
                if true_value != metric:
                    n = n + 1
                    mae = mae + np.abs(true_value - metric)
            except:
                lost = lost + 1
            
        #result.loc[criterion, 'MAE'] = mae / n
        result.loc[criterion, 'lost_rate'] = lost / total
        result.loc[criterion, 'accuracy'] = tp / total
        result.loc[criterion, 'RMSSE'] = rmsse / total
        result.index = np.arange(len(criteria))
            
    return result

In [19]:
val2test = validation2test(data[data['split']=='test'], 
                           train_hold, test_hold, ['RMSSE'])
val2test

,metric,model,accuracy,lost_rate,RMSSE
0,RMSSE,N/A,0.155172,0.0,0.887912


# Classifier solutions

In [20]:

train_80 = train_80.set_index('naming_orig')
train_hold = train_hold.set_index('naming_orig')

X_train = train_80.drop('RMSSE_model', axis=1)
y_train = train_80['RMSSE_model']

X_test = train_hold.drop('RMSSE_model', axis=1)
y_test = train_hold['RMSSE_model']

model = CatBoostClassifier(iterations=18,  depth=7, learning_rate = 0.3)
#model = CatBoostClassifier(iterations=26,  depth=8, learning_rate = 0.23)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

f1 = f1_score(y_test, y_pred, average = 'weighted')
print(f"Model f1_score on test set: {f1:.4f}")

0:	learn: 2.9688227	total: 179ms	remaining: 3.04s
1:	learn: 2.8317443	total: 228ms	remaining: 1.82s
2:	learn: 2.7079429	total: 278ms	remaining: 1.39s
3:	learn: 2.6069589	total: 329ms	remaining: 1.15s
4:	learn: 2.4909748	total: 379ms	remaining: 985ms
5:	learn: 2.4043156	total: 430ms	remaining: 859ms
6:	learn: 2.3370352	total: 478ms	remaining: 752ms
7:	learn: 2.2743725	total: 526ms	remaining: 657ms
8:	learn: 2.2175726	total: 572ms	remaining: 572ms
9:	learn: 2.1703629	total: 622ms	remaining: 498ms
10:	learn: 2.1244625	total: 673ms	remaining: 428ms
11:	learn: 2.0676530	total: 731ms	remaining: 366ms
12:	learn: 2.0272019	total: 777ms	remaining: 299ms
13:	learn: 1.9866064	total: 823ms	remaining: 235ms
14:	learn: 1.9378345	total: 869ms	remaining: 174ms
15:	learn: 1.9012896	total: 915ms	remaining: 114ms
16:	learn: 1.8630139	total: 963ms	remaining: 56.7ms
17:	learn: 1.8295221	total: 1.01s	remaining: 0us
Model f1_score on test set: 0.1765


In [21]:

def give_results(raw_data, test, predict, criteria):
    result = pd.DataFrame(index=criteria,
                          columns=['metric', 'model', 'accuracy', 'lost_rate', 'RMSSE'])
    assert np.array(test).shape == np.array(predict).shape
    result['metric'] = criteria
    result['model'] = 'N/A'
    
    total = len(test)
    for criterion in criteria:
        # Calculate accuarcy
        tp = test[test[criterion+'_model']==predict[criterion+'_model']].count().iloc[0]
        
        n = 0; mae = 0; lost = 0; rmsse = 0

        for ts in test.index:

            pred_model = predict.loc[ts, criterion+'_model']
            batch = raw_data[raw_data['naming_orig']==ts]
            try:
                true_value = batch.iloc[batch[criterion].argmin()][criterion]
                pred_value = batch[batch['model_name']==pred_model][criterion].iloc[0]
                rmsse = rmsse + pred_value
                if true_value != pred_value:
                    n = n + 1
                    mae = mae + np.abs(true_value - pred_value)
                    
            except:
                lost = lost + 1
            
        #result.loc[criterion, 'MAE'] = mae / n
        result.loc[criterion, 'lost_rate'] = lost / total
        result.loc[criterion, 'accuracy'] = tp / total
        result.loc[criterion, 'RMSSE'] = rmsse / total
        result.index = np.arange(len(criteria))
            
    return result

In [22]:
pred = y_test.reset_index(drop=False)
pred['RMSSE_model'] = y_pred[:,0]
pred = pred.set_index('naming_orig')
y_test = y_test.to_frame('RMSSE_model')

In [23]:
exp_4 = give_results(data[data['split']=='validation'], y_test, pred, ['RMSSE'])
exp_4

,metric,model,accuracy,lost_rate,RMSSE
0,RMSSE,N/A,0.241379,0.005747,0.741224


In [24]:
test_hold = test_hold.set_index('naming_orig')


X_train = train_80.drop('RMSSE_model', axis=1)
y_train = train_80['RMSSE_model']

X_test = test_hold.drop('RMSSE_model', axis=1)
y_true = test_hold['RMSSE_model']


model = CatBoostClassifier(iterations=18,  depth=7, learning_rate = 0.3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

f1_t = f1_score(y_true, y_pred, average = 'weighted')
print(f"Model f1 score on test set: {f1_t:.4f}")
#model = XGBClassifier(n_estimators=7, max_depth=2, objective='multi:softmax', learning_rate = 0.1)

0:	learn: 2.9688227	total: 45ms	remaining: 765ms
1:	learn: 2.8317443	total: 90.7ms	remaining: 726ms
2:	learn: 2.7079429	total: 139ms	remaining: 693ms
3:	learn: 2.6069589	total: 185ms	remaining: 649ms
4:	learn: 2.4909748	total: 233ms	remaining: 606ms
5:	learn: 2.4043156	total: 280ms	remaining: 560ms
6:	learn: 2.3370352	total: 331ms	remaining: 520ms
7:	learn: 2.2743725	total: 384ms	remaining: 479ms
8:	learn: 2.2175726	total: 430ms	remaining: 430ms
9:	learn: 2.1703629	total: 477ms	remaining: 382ms
10:	learn: 2.1244625	total: 525ms	remaining: 334ms
11:	learn: 2.0676530	total: 576ms	remaining: 288ms
12:	learn: 2.0272019	total: 623ms	remaining: 240ms
13:	learn: 1.9866064	total: 673ms	remaining: 192ms
14:	learn: 1.9378345	total: 723ms	remaining: 145ms
15:	learn: 1.9012896	total: 769ms	remaining: 96.1ms
16:	learn: 1.8630139	total: 815ms	remaining: 47.9ms
17:	learn: 1.8295221	total: 860ms	remaining: 0us
Model f1 score on test set: 0.1687


In [25]:
pred = y_true.reset_index(drop=False)
pred['RMSSE_model'] = y_pred[:,0]
pred = pred.set_index('naming_orig')
y_true = y_true.to_frame('RMSSE_model')

In [26]:
exp_5 = give_results(data[data['split']=='test'], y_true, pred, ['RMSSE'])
exp_5

,metric,model,accuracy,lost_rate,RMSSE
0,RMSSE,N/A,0.241379,0.0,0.906462
